In [7]:
import tensorflow as tf
import numpy as np
from transformers import BertTokenizer, TFBertModel
import pandas as pd
import openpyxl

# Load the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertModel.from_pretrained(model_name)

# Read the input data from the Excel file
data = pd.read_excel('outpute.xlsx')

# Get the sentences from the 'Cleaned Text' column
sentences = data['Cleaned Text'].tolist()

# Function to get BERT embeddings for a sentence
def get_sentence_embeddings(sentence):
    # Tokenize the sentence and convert to input tensors
    tokens = tokenizer.encode(sentence, add_special_tokens=True)
    input_ids = tf.constant([tokens])

    # Get the BERT embeddings
    outputs = model(input_ids)
    embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1)

    return embeddings

# Get BERT embeddings for each sentence in the dataset
embeddings = []
for sentence in sentences:
    embeddings.append(get_sentence_embeddings(sentence))

# Create a new Excel workbook
workbook = openpyxl.Workbook()
sheet = workbook.active

# Write the embeddings to the Excel file
row_num = 1
for row, embedding in enumerate(embeddings, start=1):
    flattened_embedding = np.ndarray.flatten(embedding.numpy())
    for col, value in enumerate(flattened_embedding, start=1):
        sheet.cell(row=row_num, column=col, value=value.item())
    row_num += 1

# Save the Excel file
workbook.save('embeddings_bert.xlsx')



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch transformers


Note: you may need to restart the kernel to use updated packages.


In [17]:
import tensorflow as tf
import numpy as np
from transformers import TFBertForSequenceClassification, BertTokenizer


In [18]:
data = pd.read_excel('outpute.xlsx')
sentences = data['Cleaned Text'].tolist()
labels = data['task_1'].tolist()


In [19]:
labels = [0 if label == 'NOT' else 1 for label in labels]
labels = np.array(labels)


In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='tf')


C:\Users\Dinesh Reddy\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dinesh Reddy\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [22]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# Convert the encoded inputs to a hashable form
encoded_inputs = (encoded_inputs["input_ids"], encoded_inputs["token_type_ids"], encoded_inputs["attention_mask"])

# Convert the labels to numpy array
labels = np.array(labels)

# Compile and fit the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])
model.fit(encoded_inputs, labels, batch_size=16, epochs=3)


Epoch 1/3
292/292 [==============================] - 3821s 13s/step - loss: 0.6450 - accuracy: 0.6146
Epoch 2/3
292/292 [==============================] - 4179s 14s/step - loss: 0.4931 - accuracy: 0.7676
Epoch 3/3
292/292 [==============================] - 3200s 11s/step - loss: 0.4248 - accuracy: 0.8049


In [32]:
# Generate predictions using the trained model
test_predictions = model.predict(test_encoded_inputs)
test_predicted_labels = np.argmax(test_predictions.logits, axis=1)

# Calculate metrics
accuracy = accuracy_score(test_labels, test_predicted_labels)
precision = precision_score(test_labels, test_predicted_labels, average='weighted')
f1 = f1_score(test_labels, test_predicted_labels, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)


146/146 [==============================] - 1123s 8s/step
Accuracy: 0.8478027867095391
Precision: 0.8498140088503491
F1 Score: 0.8479319091719225
